# 01. Prétraitement des données AMG

Ce notebook sert à charger, filtrer et extraire les caractéristiques des signaux myo-acoustiques (AMG) pour le projet de prothèse de main. 

Les étapes suivantes seront réalisées :
1. Charger les signaux bruts depuis les fichiers CSV.
2. Visualiser les signaux pour vérifier leur qualité.
3. Appliquer les transformations suivantes :
   - **Filtrage** : éliminer les bruits de fond.
   - **Extraction des caractéristiques DWT** (Décomposition en Ondelette).
   - **Extraction des caractéristiques MFCC** (Mel-Frequency Cepstral Coefficients).
4. Sauvegarder les données traitées dans un fichier CSV structuré.
